In [1]:
import os
import pickle as pk
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import numpy.linalg as la
import math
import os
import tgcn

from tgcn import TGCN, TGCN2
from load_data_tgcn import get_normalized_adj,preprocess_data,load_sz_data,load_los_data,load_pemsbay_data

from sklearn.metrics import mean_squared_error,mean_absolute_error
#import matplotlib.pyplot as plt
import time

time_start = time.time()
###### Settings ######

model_name = "tgcn"  ## 'tgcn'
data_name = "pemsbay" ## 'sz or los.'
train_rate = 0.8 ## 'rate of training set.'
seq_len = 12 ## 'time length of inputs.'
output_dim = pre_len = 3 ##'time length of prediction.'
batch_size = 24 ##'batch size.'
lr = 0.001 ##'Initial learning rate.'
training_epoch = 1001 ##'Number of epochs to train.'
gru_units = 100 ##'hidden units of gru.'

# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
#     device = torch.device('cpu')

os.environ['CUDA_VISIBLE_DEVICES']='3'
device = torch.device('cuda')


###### load data ######
if data_name == 'pemsbay':
    data, adj = load_pemsbay_data('pemsbay')

# if data_name == 'sz':
#     data, adj = load_sz_data('sz')
# if data_name == 'los':
#     data, adj = load_los_data('los')

time_len = data.shape[0]  #图的数量
num_nodes = data.shape[1]  #每个图的节点个数
data1 =np.mat(data,dtype=np.float32)  #将panda数据转化为numpy矩阵

#### normalization
max_value = np.max(data1)
data1  = data1/max_value   #将所有数据除以节点的最大值
trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len)

# trainX形状为[len(train_data) - seq_len - pre_len,seq_len,num_nodes]
# trainY形状为[len(train_data) - seq_len - pre_len,pre_len,num_nodes]

totalbatch = int(trainX.shape[0]/batch_size) #batch的数量
training_data_count = len(trainX)   ##训练数据的数量

In [4]:
trainX.shape

torch.Size([41687, 12, 325])

In [5]:
testX.shape

torch.Size([10411, 12, 325])